In [69]:
import torch
import deepspeed

In [70]:
from accelerate import Accelerator
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, Dataset
from datasets import load_dataset, load_from_disk
from tqdm.auto import tqdm
import deepspeed

In [71]:
from deepspeed.utils.zero_to_fp32 import get_fp32_state_dict_from_zero_checkpoint

In [72]:
from peft import LoraConfig, get_peft_model

In [73]:
pathPeftModel = "/cluster/home/terjenf/norwAI_All/results/Checkpoints_NRK_Peft_NorMistral/checkpoint-28000"

In [74]:
path_deep_model = "/cluster/home/terjenf/norwAI_All/results/Checkpoints_NRK_Peft_NorMistral/checkpoint-28500/global_step28500"

In [76]:
lora_state_dict = get_fp32_state_dict_from_zero_checkpoint(pathPeftModel)

Processing zero checkpoint '/cluster/home/terjenf/norwAI_All/results/Checkpoints_NRK_Peft_NorMistral/checkpoint-28000/global_step28000'


Detected checkpoint of type zero stage ZeroStageEnum.weights, world_size: 4
Parsing checkpoint created by deepspeed==0.14.2
Reconstructed Trainable fp32 state dict with 128 params 6815744 elements


In [77]:
model_id = "NorLLM-AI/NorMistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id,device_map='auto',torch_dtype=torch.bfloat16)                                         
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [78]:
model.active_adapters()

ValueError: No adapter loaded. Please load an adapter first.

In [79]:
model = PeftModel.from_pretrained(model, pathPeftModel)

SafetensorError: Error while deserializing header: InvalidHeaderDeserialization

In [81]:
base_state_dict = model.state_dict()
len(base_state_dict), type(base_state_dict), 

(419, collections.OrderedDict)

In [84]:
for state in base_state_dict.keys():
    print(base_state_dict.get(state).shape, state)


torch.Size([67993, 4096]) model.embed_tokens.weight
torch.Size([4096, 4096]) model.layers.0.self_attn.q_proj.base_layer.weight
torch.Size([16, 4096]) model.layers.0.self_attn.q_proj.lora_A.default.weight
torch.Size([4096, 16]) model.layers.0.self_attn.q_proj.lora_B.default.weight
torch.Size([1024, 4096]) model.layers.0.self_attn.k_proj.weight
torch.Size([1024, 4096]) model.layers.0.self_attn.v_proj.base_layer.weight
torch.Size([16, 4096]) model.layers.0.self_attn.v_proj.lora_A.default.weight
torch.Size([1024, 16]) model.layers.0.self_attn.v_proj.lora_B.default.weight
torch.Size([4096, 4096]) model.layers.0.self_attn.o_proj.weight
torch.Size([14336, 4096]) model.layers.0.mlp.gate_proj.weight
torch.Size([14336, 4096]) model.layers.0.mlp.up_proj.weight
torch.Size([4096, 14336]) model.layers.0.mlp.down_proj.weight
torch.Size([4096]) model.layers.0.input_layernorm.weight
torch.Size([4096]) model.layers.0.post_attention_layernorm.weight
torch.Size([4096, 4096]) model.layers.1.self_attn.q_pro

In [82]:
sata = model.get_adapter_state_dict()

ValueError: No adapter loaded. Please load an adapter first.

In [ ]:
model.load_adapter(pathPeftModel, "zero3")

In [90]:
model.active_adapters()

['zero3']

In [91]:
model.set_adapter("zero3")

In [94]:
model_state_dict = model.state_dict()

In [101]:
for state in model_state_dict.keys():
    try: 
        print("---------Nice--------")
        print(lora_state_dict.get(state).shape, state)
    except Exception as error:
        print("--------------error--------", error)
        print("--error-state-----------------------"state)


---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.embed_tokens.weight
---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.layers.0.self_attn.q_proj.base_layer.weight
---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.layers.0.self_attn.q_proj.lora_A.default.weight
---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.layers.0.self_attn.q_proj.lora_A.zero3.weight
---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.layers.0.self_attn.q_proj.lora_B.default.weight
---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.layers.0.self_attn.q_proj.lora_B.zero3.weight
---------Nice--------
--------------error-------- 'NoneType' object has no attribute 'shape'
model.layers.0.self_attn.k_proj.weight
---------Nice--------


In [93]:
adapter_state_dict = model.get_adapter_state_dict()

/cluster/home/terjenf/.conda/envs/vgdebatt/lib/python3.11/site-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


/cluster/home/terjenf/.conda/envs/vgdebatt/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [51]:
model.add_adapters(lora_state_dict, wegith_name="zero3")

AttributeError: 'MistralForCausalLM' object has no attribute 'add_adapters'

In [38]:
len(lora_state_dict)

128

In [37]:
len(base_state_dict)

291

In [42]:
for state in lora_state_dict.keys():
    print(lora_state_dict.get(state).shape, state)


torch.Size([16, 4096]) base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
torch.Size([4096, 16]) base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
torch.Size([16, 4096]) base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
torch.Size([1024, 16]) base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
torch.Size([16, 4096]) base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
torch.Size([4096, 16]) base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
torch.Size([16, 4096]) base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight
torch.Size([1024, 16]) base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight
torch.Size([16, 4096]) base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight
torch.Size([4096, 16]) base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight
torch.Size([16, 4096]) base_model.model.model.layers.2.self_

torch.Size([67993, 4096]) model.embed_tokens.weight
torch.Size([4096, 4096]) model.layers.0.self_attn.q_proj.weight
torch.Size([1024, 4096]) model.layers.0.self_attn.k_proj.weight
torch.Size([1024, 4096]) model.layers.0.self_attn.v_proj.weight
torch.Size([4096, 4096]) model.layers.0.self_attn.o_proj.weight
torch.Size([14336, 4096]) model.layers.0.mlp.gate_proj.weight
torch.Size([14336, 4096]) model.layers.0.mlp.up_proj.weight
torch.Size([4096, 14336]) model.layers.0.mlp.down_proj.weight
torch.Size([4096]) model.layers.0.input_layernorm.weight
torch.Size([4096]) model.layers.0.post_attention_layernorm.weight
torch.Size([4096, 4096]) model.layers.1.self_attn.q_proj.weight
torch.Size([1024, 4096]) model.layers.1.self_attn.k_proj.weight
torch.Size([1024, 4096]) model.layers.1.self_attn.v_proj.weight
torch.Size([4096, 4096]) model.layers.1.self_attn.o_proj.weight
torch.Size([14336, 4096]) model.layers.1.mlp.gate_proj.weight
torch.Size([14336, 4096]) model.layers.1.mlp.up_proj.weight
torch.S

In [34]:
len(base_state_dict.keys())

291

In [29]:

for adapter in state_dict.keys():
    print(state_dict.get(adapter))
    model.set_adapter(state_dict.get(adapter))

tensor([[-0.0099,  0.0508, -0.0007,  ..., -0.0313, -0.0046,  0.0463],
        [ 0.0074, -0.0108, -0.0074,  ...,  0.0257,  0.0021,  0.0282],
        [ 0.0165, -0.0406, -0.0053,  ..., -0.0101,  0.0094, -0.0284],
        ...,
        [-0.0180,  0.0013,  0.0021,  ...,  0.0304, -0.0095, -0.0048],
        [-0.0029,  0.0155, -0.0128,  ...,  0.0160, -0.0106, -0.0380],
        [-0.0062,  0.0096, -0.0147,  ..., -0.0474,  0.0018, -0.0128]],
       grad_fn=<ViewBackward0>)


AttributeError: 'MistralForCausalLM' object has no attribute 'peft_config'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for MistralForCausalLM:
	Missing key(s) in state_dict: "model.embed_tokens.weight", "model.layers.0.self_attn.q_proj.weight", "model.layers.0.self_attn.k_proj.weight", "model.layers.0.self_attn.v_proj.weight", "model.layers.0.self_attn.o_proj.weight", "model.layers.0.mlp.gate_proj.weight", "model.layers.0.mlp.up_proj.weight", "model.layers.0.mlp.down_proj.weight", "model.layers.0.input_layernorm.weight", "model.layers.0.post_attention_layernorm.weight", "model.layers.1.self_attn.q_proj.weight", "model.layers.1.self_attn.k_proj.weight", "model.layers.1.self_attn.v_proj.weight", "model.layers.1.self_attn.o_proj.weight", "model.layers.1.mlp.gate_proj.weight", "model.layers.1.mlp.up_proj.weight", "model.layers.1.mlp.down_proj.weight", "model.layers.1.input_layernorm.weight", "model.layers.1.post_attention_layernorm.weight", "model.layers.2.self_attn.q_proj.weight", "model.layers.2.self_attn.k_proj.weight", "model.layers.2.self_attn.v_proj.weight

In [ ]:
    config = LoraConfig(
    r=rank, #attention heads, rank of the attention matrix, i think
    lora_alpha= lora_alpha, #alpha scaling, scaling factor for the weight matrices
    # target_modules=["q_proj", "v_proj"], #will be set after i know the names
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
    )

In [ ]:
model = PeftModel.from_pretrained(model, pathPeftModel)
print("Peft Model : ", model.device)
print(f"Running merge_and_unload")
model = model.merge_and_unload()

In [41]:
tokenizer.pad_token = tokenizer.eos_token

In [6]:
deepspeed.init_distributed(dist_backend="nccl", world_size=2)

[2024-07-05 01:43:00,119] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-07-05 01:43:00,119] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-07-05 01:43:00,195] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.1.4.1, master_port=29500
[2024-07-05 01:43:00,195] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [7]:
# Initialize the DeepSpeed-Inference engine
ds_engine = deepspeed.init_inference(model,
                                 tensor_parallel={"tp_size": 1},
                                 dtype=torch.half,

                                 replace_with_kernel_inject=True)


[2024-07-05 01:50:34,460] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.2, git-hash=unknown, git-branch=unknown
[2024-07-05 01:50:34,462] [INFO] [logging.py:96:log_dist] [Rank 0] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


You shouldn't move a model when it is dispatched on multiple devices.


In [18]:
tokens = tokenizer("hei, jeg heter terje", max_length=20, return_tensors='pt')
tokens

{'input_ids': tensor([[    1, 36512, 28725, 32008, 23402,  3636,  2099]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [19]:
type(model)

transformers.models.mistral.modeling_mistral.MistralForCausalLM

In [27]:
model.hf_device_map

OrderedDict([('model.embed_tokens', 0),
             ('model.layers.0', 0),
             ('model.layers.1', 0),
             ('model.layers.2', 0),
             ('model.layers.3', 0),
             ('model.layers.4', 0),
             ('model.layers.5', 0),
             ('model.layers.6', 0),
             ('model.layers.7', 0),
             ('model.layers.8', 0),
             ('model.layers.9', 0),
             ('model.layers.10', 0),
             ('model.layers.11', 0),
             ('model.layers.12', 0),
             ('model.layers.13', 0),
             ('model.layers.14', 0),
             ('model.layers.15', 0),
             ('model.layers.16', 1),
             ('model.layers.17', 1),
             ('model.layers.18', 1),
             ('model.layers.19', 1),
             ('model.layers.20', 1),
             ('model.layers.21', 1),
             ('model.layers.22', 1),
             ('model.layers.23', 1),
             ('model.layers.24', 1),
             ('model.layers.25', 1),
        

In [40]:
model.generate(input_ids=tokens['input_ids'].to("cuda:0"), attention_mask=tokens['attention_mask'].to("cuda:0"))
print("hei")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [38]:
model(input_ids=tokens['input_ids'].to("cuda:0"), attention_mask=tokens['attention_mask'].to("cuda:0"), labels=tokens['input_ids'].to("cuda:1"))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [33]:
model(input_ids=tokens['input_ids'].to(torch.device('cuda')), attention_mask=tokens['attention_mask'].to(torch.device('cuda')))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [14]:
type(ds_engine)

deepspeed.inference.engine.InferenceEngine

In [8]:
deepspeed_path = "/cluster/home/terjenf/norwAI_All/results/Checkpoints_NRK_Peft_NorMistral/checkpoint-28000"

In [48]:
ds_engine.load_checkpoint(deepspeed_path)

device(type='cuda', index=0)

In [49]:
ds_engine.module.device

device(type='cuda', index=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': [1, 36512, 28725, 32008, 23402, 3636, 2099], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [10]:
model(input_ids=tokens['input_ids'], tokens['attention_mask'])

SyntaxError: positional argument follows keyword argument (1990286007.py, line 1)

In [40]:
detokenized = tokenizer.decode(tokens['input_ids'])
detokenized

'<s> hei, jeg heter terje'

In [41]:
output = model(tokens)

AttributeError: 